In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

#--------------------------------------------------
# Modify the model by increasing the number of neurons
class ModifiedNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 1024),  
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
#---------------------------------------------------

model = ModifiedNeuralNetwork().to(device)
print(model)

Using cpu device
ModifiedNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
 loss_fn = nn.CrossEntropyLoss()

# I tried to change the error function CrossEntropyLoss() to MSELoss(), but it gives an error, so I left it as it is
#loss_fn = nn.MSELoss()  # або інша функція

# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# Changing the speed of learning
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
epochs = 5  # Кількість епох
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.291129  [   64/60000]
loss: 2.115433  [ 6464/60000]
loss: 1.694287  [12864/60000]
loss: 1.435980  [19264/60000]
loss: 1.101041  [25664/60000]
loss: 1.017418  [32064/60000]
loss: 0.995682  [38464/60000]
loss: 0.861094  [44864/60000]
loss: 0.855400  [51264/60000]
loss: 0.810783  [57664/60000]
Test Error: 
 Accuracy: 71.8%, Avg loss: 0.784657 

Epoch 2
-------------------------------
loss: 0.777404  [   64/60000]
loss: 0.857311  [ 6464/60000]
loss: 0.578409  [12864/60000]
loss: 0.785564  [19264/60000]
loss: 0.651312  [25664/60000]
loss: 0.634623  [32064/60000]
loss: 0.713584  [38464/60000]
loss: 0.687372  [44864/60000]
loss: 0.687224  [51264/60000]
loss: 0.636797  [57664/60000]
Test Error: 
 Accuracy: 78.4%, Avg loss: 0.623593 

Epoch 3
-------------------------------
loss: 0.548270  [   64/60000]
loss: 0.654196  [ 6464/60000]
loss: 0.434793  [12864/60000]
loss: 0.669549  [19264/60000]
loss: 0.564366  [25664/60000]
loss: 0.554680  [32064/600

In [7]:
# The accuracy of the model improved, but the training time remained about the same.

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [8]:
# model = NeuralNetwork().to(device)
# model.load_state_dict(torch.load("model.pth", weights_only=True))
#-----------------------------------------------------------------
model = ModifiedNeuralNetwork().to(device)  # або ваша модель
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [9]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
